In [75]:
import subprocess
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import ParameterGrid
import os
import shutil
from sklearn.metrics import roc_auc_score


output_qza='/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/otu_table1.qza'
metadata="/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/meta_table1.tsv"
ancomoutput='/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza'
exported='/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/exported_ancompd'
truedaa="/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/truedaa.tsv"
simulated="/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/otu_table1.tsv"
otu_tab_sim= pd.read_csv(simulated,sep="\t", index_col=0)
tr = pd.read_csv(truedaa,sep="\t", index_col=0)
# Define a function to run QIIME 2's ANCOM-BC with system calls
def run_ancombc(table_path, metadata_path, formula, max_iter, tol, alpha, lib_cut, output_path):
    cmd = [
        "qiime", "composition", "ancombc",
        "--i-table", table_path,
        "--m-metadata-file", metadata_path,
        "--p-formula", formula,
       # "--p-p-adj-method", p_adj_method,
        "--p-max-iter", str(max_iter),
        "--p-tol", str(tol),
        "--p-alpha", str(alpha),
        "--p-lib-cut", str(lib_cut),
        "--o-differentials", output_path
    ]
    subprocess.run(cmd, check=True)

# Define the parameter grid for tuning
param_grid = {
    #'p_adj_method': ['holm', 'hochberg', 'hommel', 'bonferroni', 'BH', 'BY', 'fdr'],
     'tol' : [1e-03, 1e-04, 1e-05, 1e-06],
     'max_iter': [20, 40, 50, 75, 100, 125, 150, 175, 200],
   #  'zero_cut' : np.arange(0.1, 1.1, 0.1),
     'alpha' : np.arange(0.01, 0.08, 0.01),
     'lib_cut' : np.arange(0, 100, 10)
}

# Convert parameter grid to list of dictionaries
param_list = list(ParameterGrid(param_grid))

# Placeholder for results
results = []
k=0
# Run tuning
for params in param_list:
    try:
        run_ancombc(
            table_path=output_qza,
            metadata_path=metadata,
            formula='covariate',
            #p_adj_method=params['p_adj_method'],
            
            max_iter=params['max_iter'],
            tol=params['tol'],
           # zero_cut=params['zero_cut'],
            alpha=params['alpha'],
            lib_cut=params['lib_cut'],
            output_path=ancomoutput
        )

        # Export the results from QIIME2
        subprocess.run([
            'qiime', 'tools', 'export',
            '--input-path', ancomoutput,
            '--output-path', exported
        ], check=True)

        # Construct file path for q_val_slice.csv
        q_val_csv = os.path.join(exported, 'q_val_slice.csv')

        # Check if the file exists before reading
        if not os.path.exists(q_val_csv):
            print(f"Expected q_val_slice.csv does not exist at path: {q_val_csv}")
        else:
            # Read the q-value results
            ancom_q_val = pd.read_csv(q_val_csv, index_col=0)

            # take only necessary data
            ancom_q_val = ancom_q_val[['covariateunhealthy']]
    
            # construct output dataframe
            otu_tab_sim_index = pd.Index(otu_tab_sim.index)  # assuming otu_tab_sim is already defined
            missing = ~otu_tab_sim_index.isin(ancom_q_val.index)
            missing_index = otu_tab_sim_index[missing]
            nqv = pd.DataFrame({'covariateunhealthy': [None] * len(missing_index)}, index=missing_index)
            q_val_all = pd.concat([ancom_q_val, nqv])
            q_val_all.rename(columns={'covariateunhealthy': 'q_value'}, inplace=True)
            pred = (ancom_q_val['covariateunhealthy'] < 0.05).astype(int).replace({0: 'NOT_DA', 1: 'DA'})
            pred_df = pd.DataFrame({'pred': pred})
    
            # Assume tr is already defined and is a DataFrame
            out_df = pd.concat([pred_df, q_val_all, tr], axis=1)

            # Calculate metrics
            out_df['truth'] = out_df['truth'].apply(lambda x: 1 if x == 'DA' else 0)
            out_df['pred'] = out_df['pred'].apply(lambda x: 1 if x == 'DA' else 0)
            FP = sum((out_df['truth'] == 0) & (out_df['pred'] == 1))
            FN = sum((out_df['truth'] == 1) & (out_df['pred'] == 0))
            TP = sum((out_df['truth'] == 1) & (out_df['pred'] == 1))
            TN = sum((out_df['truth'] == 0) & (out_df['pred'] == 0))

            precision = TP / (TP + FP) if TP + FP > 0 else None
            recall = TP / (TP + FN) if TP + FN > 0 else None
            fpr = FP / (FP + TN) if FP + TN > 0 else None
            f1_score = 2 * (recall * precision) / (precision + recall) if precision + recall > 0 else None
            accuracy = (TP + TN) / (TP + TN + FP + FN)
            performance_score = f1_score
            # Calculate AUC-ROC
            # Ensure 'truth' is binary and 'pred_prob' is the probability of being class 1 (DA)
            out_df.fillna(0, inplace=True)
            if 'q_value' in out_df.columns and out_df['truth'].isin([0, 1]).all():
                auc_roc = roc_auc_score(out_df['truth'], out_df['q_value'])
            else:
                auc_roc = None  # or some default value if probabilities are not available

            
            
            results.append((params, auc_roc))
            # deleting file
            directory='/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result'
            if not os.path.exists(directory):
                print(f"Directory does not exist: {directory}")
                

            # Iterate over all the files and directories in the given directory
            for filename in os.listdir(directory):
                file_path = os.path.join(directory, filename)

                try:
                    # If it's a file, delete it
                    if os.path.isfile(file_path) or os.path.islink(file_path):
                        os.unlink(file_path)
                      # If it's a directory, delete the directory and all its contents
                    elif os.path.isdir(file_path):
                        shutil.rmtree(file_path)
                except Exception as e:
                    print(f"Failed to delete {file_path}. Reason: {e}")
            
        k=k+1
        if(k>=len(param_list)):
        #if(k>=1):   
            break
    except subprocess.CalledProcessError as e:
        print(f"Error running ANCOM-BC: {e}")

# Convert results to DataFrame
results_df = pd.DataFrame(results, columns=['params', 'performance_score'])

# Extract the hyperparameters and scores
X = results_df['params'].apply(pd.Series)
y = results_df['performance_score']
print(X)
print(y)
# Fit the Decision Tree Regressor
regressor = DecisionTreeRegressor()
regressor.fit(X, y)

# Now you can analyze the regressor to find out which hyperparameters are important
print(regressor.feature_importances_)

# Predict with the regressor to find the best hyperparameters
best_params_index = np.argmax(regressor.predict(X))
best_params = X.iloc[best_params_index]
print(f"Best hyperparameters: {best_params.to_dict()}")


Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataLoafPackageDirFmt to directory /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/exported_ancompd
Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataLoafPackageDirFmt to directory /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/exported_ancompd
Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataL

Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataLoafPackageDirFmt to directory /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/exported_ancompd
Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataLoafPackageDirFmt to directory /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/exported_ancompd
Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataL

Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataLoafPackageDirFmt to directory /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/exported_ancompd
Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataLoafPackageDirFmt to directory /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/exported_ancompd
Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataL

Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataLoafPackageDirFmt to directory /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/exported_ancompd
Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataLoafPackageDirFmt to directory /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/exported_ancompd
Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataL

Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataLoafPackageDirFmt to directory /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/exported_ancompd
Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataLoafPackageDirFmt to directory /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/exported_ancompd
Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataL

...

Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataLoafPackageDirFmt to directory /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/exported_ancompd
Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataLoafPackageDirFmt to directory /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/exported_ancompd
Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataL

Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataLoafPackageDirFmt to directory /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/exported_ancompd
Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataLoafPackageDirFmt to directory /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/exported_ancompd
Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataL

Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataLoafPackageDirFmt to directory /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/exported_ancompd
Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataLoafPackageDirFmt to directory /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/exported_ancompd
Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataL

Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataLoafPackageDirFmt to directory /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/exported_ancompd
Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataLoafPackageDirFmt to directory /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/exported_ancompd
Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataL

Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataLoafPackageDirFmt to directory /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/exported_ancompd
Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataLoafPackageDirFmt to directory /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/exported_ancompd
Saved FeatureData[DifferentialAbundance] to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza as DataL

In [17]:
#Best hyperparameters: {'alpha': 0.01, 'lib_cut': 0.0, 'max_iter': 50.0, 'tol': 0.001}
#Best hyperparameters: {'alpha': 0.01, 'lib_cut': 0.0, 'max_iter': 50.0, 'tol': 0.001}
#Best hyperparameters: {'alpha': 0.01, 'lib_cut': 0.0, 'max_iter': 20.0, 'tol': 1e-05}
#Best hyperparameters: {'alpha': 0.01, 'lib_cut': 0.0, 'max_iter': 75.0, 'tol': 1e-05}
#Best hyperparameters: {'alpha': 0.01, 'lib_cut': 50.0, 'max_iter': 75.0, 'tol': 1e-05}
#Best hyperparameters: {'alpha': 0.01, 'lib_cut': 0.0, 'max_iter': 75.0, 'tol': 0.001}
#Best hyperparameters: {'alpha': 0.01, 'lib_cut': 0.0, 'max_iter': 75.0, 'tol': 0.001}

In [73]:
FN

110

In [18]:
# Define the parameter grid for tuning
param_grid = {
    #'p_adj_method': ['holm', 'hochberg', 'hommel', 'bonferroni', 'BH', 'BY', 'fdr'],
     'tol' : [1e-05,1e-03, 1e-04, 1e-05, 1e-06],
     'max_iter': [20, 40, 50, 75, 100, 125, 150, 175, 200],
   #  'zero_cut' : np.arange(0.1, 1.1, 0.1),
     'alpha' : np.arange(0.01, 0.05, 0.01),
     'lib_cut' : np.arange(0, 100, 10)
}

# Convert parameter grid to list of dictionaries
param_list = list(ParameterGrid(param_grid))

In [22]:
len(param_list)

1800

In [10]:
from sklearn.metrics import roc_auc_score


In [14]:
# Calculate AUC-ROC
# Ensure 'truth' is binary and 'pred_prob' is the probability of being class 1 (DA)
out_df.fillna(0, inplace=True)
if 'q_value' in out_df.columns and out_df['truth'].isin([0, 1]).all():
    auc_roc = roc_auc_score(out_df['truth'], out_df['q_value'])
else:
    auc_roc = None  # or some default value if probabilities are not available


In [15]:
auc_roc

0.6933389473684209

In [2]:
X


,alpha,lib_cut,max_iter,tol
0,0.01,0.0,20.0,0.001000
1,0.01,0.0,20.0,0.000100
2,0.01,0.0,20.0,0.000010
3,0.01,0.0,20.0,0.000001
4,0.01,0.0,40.0,0.001000
...,...,...,...,...
2875,0.08,90.0,175.0,0.000001
2876,0.08,90.0,200.0,0.001000
2877,0.08,90.0,200.0,0.000100
2878,0.08,90.0,200.0,0.000010


In [3]:
y

0       0.693339
1       0.693339
2       0.693339
3       0.693339
4       0.693342
          ...   
2875    0.693344
2876    0.693344
2877    0.693344
2878    0.693344
2879    0.693344
Name: performance_score, Length: 2880, dtype: float64

In [4]:
 out_parameter_f1 = pd.concat([X, y], axis=1)

In [6]:
 out_parameter_f1

,alpha,lib_cut,max_iter,tol,performance_score
0,0.01,0.0,20.0,0.001000,0.693339
1,0.01,0.0,20.0,0.000100,0.693339
2,0.01,0.0,20.0,0.000010,0.693339
3,0.01,0.0,20.0,0.000001,0.693339
4,0.01,0.0,40.0,0.001000,0.693342
...,...,...,...,...,...
2875,0.08,90.0,175.0,0.000001,0.693344
2876,0.08,90.0,200.0,0.001000,0.693344
2877,0.08,90.0,200.0,0.000100,0.693344
2878,0.08,90.0,200.0,0.000010,0.693344


In [8]:
out_parameter_f1.to_csv(p, index=False)

In [7]:
p="/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/dt_out_parameter_auc_roc.csv"

In [ ]:
#rendomsearch

In [3]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
import random

import subprocess
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import ParameterGrid
import os
import shutil


output_qza='/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/otu_table1.qza'
metadata="/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/meta_table1.tsv"
ancomoutput='/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/ancombc.qza'
exported='/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/exported_ancompd'
truedaa="/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/truedaa.tsv"
simulated="/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/otu_table1.tsv"
otu_tab_sim= pd.read_csv(simulated,sep="\t", index_col=0)
tr = pd.read_csv(truedaa,sep="\t", index_col=0)
# Define a function to run QIIME 2's ANCOM-BC with system calls
def run_ancombc(table_path, metadata_path, formula, max_iter, tol, alpha, lib_cut, output_path):
    cmd = [
        "qiime", "composition", "ancombc",
        "--i-table", table_path,
        "--m-metadata-file", metadata_path,
        "--p-formula", formula,
       # "--p-p-adj-method", p_adj_method,
        "--p-max-iter", str(max_iter),
        "--p-tol", str(tol),
        "--p-alpha", str(alpha),
        "--p-lib-cut", str(lib_cut),
        "--o-differentials", output_path
    ]
    subprocess.run(cmd, check=True)
# Assuming other necessary imports are already done

# Define the parameter space
param_distributions = {
    'tol': [1e-05, 1e-04, 1e-03, 1e-02],
    'max_iter': [20, 40, 50, 75, 100, 125, 150, 175, 200],
    'alpha': [0.01, 0.02, 0.03, 0.04, 0.05],
    'lib_cut': list(range(0, 100, 10))
}

# Function to execute ANCOM-BC and return the performance metric
def ancombc_performance_score(params):
    try:
        run_ancombc(
            table_path=output_qza,
            metadata_path=metadata,
            formula='covariate',
            max_iter=params['max_iter'],
            tol=params['tol'],
            alpha=params['alpha'],
            lib_cut=params['lib_cut'],
            output_path=ancomoutput
        )
        # Export the results from QIIME2
        subprocess.run([
            'qiime', 'tools', 'export',
            '--input-path', ancomoutput,
            '--output-path', exported
        ], check=True)

        # Construct file path for q_val_slice.csv
        q_val_csv = os.path.join(exported, 'q_val_slice.csv')

        # Check if the file exists before reading
        if not os.path.exists(q_val_csv):
            print(f"Expected q_val_slice.csv does not exist at path: {q_val_csv}")
        else:
            # Read the q-value results
            ancom_q_val = pd.read_csv(q_val_csv, index_col=0)

            # take only necessary data
            ancom_q_val = ancom_q_val[['covariateunhealthy']]
    
            # construct output dataframe
            otu_tab_sim_index = pd.Index(otu_tab_sim.index)  # assuming otu_tab_sim is already defined
            missing = ~otu_tab_sim_index.isin(ancom_q_val.index)
            missing_index = otu_tab_sim_index[missing]
            nqv = pd.DataFrame({'covariateunhealthy': [None] * len(missing_index)}, index=missing_index)
            q_val_all = pd.concat([ancom_q_val, nqv])
            q_val_all.rename(columns={'covariateunhealthy': 'q_value'}, inplace=True)
            pred = (ancom_q_val['covariateunhealthy'] < 0.05).astype(int).replace({0: 'NOT_DA', 1: 'DA'})
            pred_df = pd.DataFrame({'pred': pred})
    
            # Assume tr is already defined and is a DataFrame
            out_df = pd.concat([pred_df, q_val_all, tr], axis=1)

            # Calculate metrics
            out_df['truth'] = out_df['truth'].apply(lambda x: 1 if x == 'DA' else 0)
            out_df['pred'] = out_df['pred'].apply(lambda x: 1 if x == 'DA' else 0)
            FP = sum((out_df['truth'] == 0) & (out_df['pred'] == 1))
            FN = sum((out_df['truth'] == 1) & (out_df['pred'] == 0))
            TP = sum((out_df['truth'] == 1) & (out_df['pred'] == 1))
            TN = sum((out_df['truth'] == 0) & (out_df['pred'] == 0))

            precision = TP / (TP + FP) if TP + FP > 0 else None
            recall = TP / (TP + FN) if TP + FN > 0 else None
            fpr = FP / (FP + TN) if FP + TN > 0 else None
            f1_score = 2 * (recall * precision) / (precision + recall) if precision + recall > 0 else None
            accuracy = (TP + TN) / (TP + TN + FP + FN)

        # Your code to export results, read q-values, and calculate performance metrics goes here
        # ....

        # Return the performance metric (e.g., F1 score)
        return f1_score  # Replace with your actual performance score calculation

    except subprocess.CalledProcessError as e:
        print(f"Error running ANCOM-BC: {e}")
        return None  # In case of an error, return None or an appropriate default value

# Custom scorer for RandomizedSearchCV
def custom_scorer(estimator, X, y):
    params = estimator.get_params()
    score = ancombc_performance_score(params)
    return score

# Dummy estimator class for RandomizedSearchCV
class DummyEstimator:
    def __init__(self, **params):
        self.params = params

    def fit(self, X, y):
        pass

    def get_params(self, deep=True):
        return self.params

    def set_params(self, **parameters):
        self.params = parameters
        return self

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=DummyEstimator(),
    param_distributions=param_distributions,
    n_iter=10,  # Number of parameter settings that are sampled
    scoring=make_scorer(custom_scorer, greater_is_better=True),
    cv=[(slice(None), slice(None))],  # Dummy split to comply with the interface
    verbose=1
)

# Run RandomizedSearchCV
random_search.fit(None, None)

# Print best parameters
print("Best hyperparameters:", random_search.best_params_)

Fitting 1 folds for each of 10 candidates, totalling 10 fits


TypeError: 'NoneType' object is not subscriptable

In [91]:
  pred = pd.DataFrame({'pred': ['DA' if i else 'NOT_DA' for i in e]}, index=otu_tab_sim.index)


In [95]:
pred[pred['pred']=="DA"]


,pred


In [59]:
import subprocess
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import ParameterGrid
import os



# Define the paths to the table, metadata, and output directories
#original_data<-"~/from_pendrive/PhD/DAA_benchmark_study/qiime_ancombc/data/PD_10483/otu_pd_qiime.tsv"
#simulated_data<-"/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/otu_table1.txt"
#biom_file <- '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/otu_table1.biom'
output_qza='/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/otu_table1.qza'
metadata="/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/meta_table1.tsv"
output_flash='/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/flashnet.qza'
exported='/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/exported_flash'
neighbour='/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/neighbours1.qza'
truedaa="/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/truedaa.tsv"
simulated="/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/otu_table1.tsv"
tr = pd.read_csv(truedaa,sep="\t", index_col=0)
otu_tab_sim=pd.read_csv(simulated, sep="\t", index_col=0)
otu_tab_sim.index=otu_tab_sim["featureid"]
otu_tab_sim=otu_tab_sim.drop("featureid", axis=1)
# Define a function to run QIIME 2's ANCOM-BC with system calls
def run_flashweave(table_path, metadata_path, max_k, alpha, output_path):
    cmd = [
        "qiime", "makarsa", "flashweave",
        "--i-table", table_path,
        "--m-metadata-file", metadata_path,
        "--p-max-k", str(max_k),
        "--p-alpha", str(alpha),
        "--o-network", output_path
    ]
    subprocess.run(cmd, check=True)

# Define the parameter grid for tuning
param_grid = {
     'max_k': [0, 1, 2, 3, 4],
     'alpha' : [0.01, 0.02, 0.03, 0.04, 0.05]
     
}

# Convert parameter grid to list of dictionaries
param_list = list(ParameterGrid(param_grid))

# Placeholder for results
results = []
k=0
# Run tuning
for params in param_list:
    try:
        run_flashweave(
            table_path=output_qza,
            metadata_path=metadata,
            max_k=params['max_k'],
            alpha=params['alpha'],
            output_path=output_flash
        )
        cmd = [
            "qiime", "makarsa", "list-neighbours",
            "--i-network",output_flash,
            "--p-feature-id", "covariate",
            "--o-neighbours",neighbour
        ]
        subprocess.run(cmd, check=True)

        # Export the results from QIIME2
        subprocess.run([
            'qiime', 'tools', 'export',
            '--input-path', neighbour,
            '--output-path', exported
        ], check=True)

        # Construct file path for q_val_slice.csv
        metadata_tsv = os.path.join(exported, 'metadata.tsv')

        # Check if the file exists before reading
        if not os.path.exists(metadata_tsv):
            print(f"Expected metadata_tsv does not exist at path: {metadata_tsv}")
        else:
            # Read the q-value results
            #metadata_tsv = pd.read_csv(metadata_tsv, index_col=0)
            
            # Read the results
            neighbours = pd.read_csv(metadata_tsv, sep='\t', index_col=0)

            # Take only necessary data
            neighbours = neighbours.iloc[1:, ]

            # Assuming 'otu_tab_sim' is already defined as a DataFrame
            # Check if indices of 'otu_tab_sim' are in 'neighbours'
            e = otu_tab_sim.index.isin(neighbours.index)

            # Create a DataFrame for predictions
            pred = pd.DataFrame({'pred': ['DA' if i else 'NOT_DA' for i in e]}, index=otu_tab_sim.index)

            # Assuming 'tr' is already defined and is a DataFrame
            # Combine 'pred' and 'tr' into one DataFrame
            out_df = pd.concat([pred, tr], axis=1)

            # Calculate metrics
            out_df['truth'] = out_df['truth'].apply(lambda x: 1 if x == 'DA' else 0)
            out_df['pred'] = out_df['pred'].apply(lambda x: 1 if x == 'DA' else 0)
            FP = sum((out_df['truth'] == 0) & (out_df['pred'] == 1))
            FN = sum((out_df['truth'] == 1) & (out_df['pred'] == 0))
            TP = sum((out_df['truth'] == 1) & (out_df['pred'] == 1))
            TN = sum((out_df['truth'] == 0) & (out_df['pred'] == 0))

            precision = TP / (TP + FP) if TP + FP > 0 else None
            recall = TP / (TP + FN) if TP + FN > 0 else None
            fpr = FP / (FP + TN) if FP + TN > 0 else None
            #f1_score = 2 * (recall * precision) / (precision + recall) if precision + recall > 0 else None
            # Check if either precision or recall is None
            if precision is not None and recall is not None:
                f1_score = 2 * (recall * precision) / (precision + recall) if precision + recall > 0 else None
            else:
                f1_score = None

            
            performance_score = (TP + TN) / (TP + TN + FP + FN)
             
            results.append((params, f1_score))
            
            # deleting file
            directory='/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result'
            if not os.path.exists(directory):
                print(f"Directory does not exist: {directory}")
                

            # Iterate over all the files and directories in the given directory
            for filename in os.listdir(directory):
                file_path = os.path.join(directory, filename)

                try:
                    # If it's a file, delete it
                    if os.path.isfile(file_path) or os.path.islink(file_path):
                        os.unlink(file_path)
                      # If it's a directory, delete the directory and all its contents
                    elif os.path.isdir(file_path):
                        shutil.rmtree(file_path)
                except Exception as e:
                    print(f"Failed to delete {file_path}. Reason: {e}")
            
        k=k+1
        if(k>=1):
            break
    except subprocess.CalledProcessError as e:
        print(f"Error running flashweave: {e}")

# Convert results to DataFrame
results_df = pd.DataFrame(results, columns=['params', 'performance_score'])

# Extract the hyperparameters and scores
X = results_df['params'].apply(pd.Series)
y = results_df['performance_score']
X
y
# Fit the Decision Tree Regressor
regressor = DecisionTreeRegressor()
regressor.fit(X, y)

# Now you can analyze the regressor to find out which hyperparameters are important
print(regressor.feature_importances_)

# Predict with the regressor to find the best hyperparameters
best_params_index = np.argmax(regressor.predict(X))
best_params = X.iloc[best_params_index]
print(f"Best hyperparameters: {best_params.to_dict()}")

Saved Network to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/flashnet.qza
Saved ImmutableMetadata to: /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/neighbours1.qza
Exported /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/neighbours1.qza as ImmutableMetadataDirectoryFormat to directory /home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/exported_flash
[0. 0.]
Best hyperparameters: {'alpha': 0.01, 'max_k': 0.0}


In [13]:
#Best hyperparameters: {'alpha': 0.01, 'max_k': 0.0}

In [20]:
otu_tab_sim.index.isin(neighbours.index)

array([False, False, False, ..., False, False, False])

In [66]:
e

array([False, False, False, ..., False, False, False])

In [60]:
out_df

,pred,truth
4468234,0,0
4306262,0,0
4447072,0,1
4467447,0,0
4365130,0,0
...,...,...
715913,0,0
176298,0,0
4477696,0,0
4431178,0,0


In [21]:
 neighbours

,mv,Degree Centrality,Betweenness Centrality,Closeness Centrality,Eigenvector Centrality,Assortativity,Weight to covariate
Feature ID,,,,,,,
4469576,0,0.0634920634920635,0.00394743595653309,0.438176197836167,0.0105692861126003,64.5416666666667,-0.29021281003952
4347520,0,0.0440917107583774,0.000489408165535575,0.38584552568901,0.00247876932201871,62.26,-0.21412605047226
3903651,0,0.0423280423280423,0.0019666606319697,0.415536826676438,0.00439001901171716,61.0833333333333,-0.355087429285049
368486,0,0.0555555555555556,0.00248272040585026,0.412663755458515,0.00596786134562384,57.8888888888889,-0.401213496923447
3090117,0,0.0873015873015873,0.00595967835721541,0.447690485590209,0.00966146010085982,57.7979797979798,-0.220837697386742
4472091,0,0.0696649029982363,0.00353130354768039,0.419844502036283,0.00479645283461257,55.9367088607595,-0.228214651346207
306299,0,0.0361552028218695,0.0016656659327962,0.393340270551509,0.00180308479278047,52.219512195122,-0.199893653392792
184882,0,0.0255731922398589,0.000643140814627997,0.383756345177665,0.00215405043000887,58.9310344827586,-0.192229196429253
173965,0,0.0238095238095238,0.000236192363320391,0.373640856672158,0.00121558769561672,52.1111111111111,-0.368639796972275


In [22]:
 neighbours = neighbours.iloc[1:, ]


In [31]:
 neighbours
rows_to_remove = ['cofounder1', 'cofounder2']

# Remove the rows
df = neighbours.drop(rows_to_remove)


In [28]:
df1=df["Betweenness Centrality"]

In [35]:
df1=pd.DataFrame(df1)

In [ ]:
# deleting file
            directory='/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/result'
            if not os.path.exists(directory):
                print(f"Directory does not exist: {directory}")
                

            # Iterate over all the files and directories in the given directory
            for filename in os.listdir(directory):
                file_path = os.path.join(directory, filename)

                try:
                    # If it's a file, delete it
                    if os.path.isfile(file_path) or os.path.islink(file_path):
                        os.unlink(file_path)
                      # If it's a directory, delete the directory and all its contents
                    elif os.path.isdir(file_path):
                        shutil.rmtree(file_path)
                except Exception as e:
                    print(f"Failed to delete {file_path}. Reason: {e}")
            
            